# Installation

In [ ]:
#in vispy
! pip install vispy

In [ ]:
# installing pyvista
! pip install pyvista

# Library Import

In [ ]:
#essential (p1)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
#essential (p2)
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#extra installation libraries (install if needed)
try:
    import vispy
    from vispy import scene
    VISPY_AVAILABLE = True
except:
    VISPY_AVAILABLE = False
    print("Vispy not available - install with: pip install vispy")

try:
    import pyvista as pv
    PYVISTA_AVAILABLE = True
except:
    PYVISTA_AVAILABLE = False
    print("PyVista not available - install with: pip install pyvista")

# Dashboard Creation

## Data Loading

In [ ]:
#file path declaration (main data path)
main_path = "/kaggle/input/nfl-big-data-bowl-2026-analytics/114239_nfl_competition_files_published_analytics_final" #base path
main_train_path = f"{main_path}/train" #main train

In [ ]:
#class builder with dynamic data
class NFLDataProcessor:
    """Load and process NFL tracking data"""

    def __init__(self):
        self.field_length = 120
        self.field_width = 53.3
        self.data_loaded = False

    def load_data(self, data_path=main_train_path, week=1):
        #dynamic data builder
        try:
            if data_path:
                # conditional check in string
                if week > 18:
                    return "ERROR"
                elif week <= 18 and week < 10 :
                    week_str = f"{str(week:02d)}"
                else :
                    week_str = str(week)

                # Build dynamic filenames
                input_file = f"{data_path}/input_2023_w{week_str}.csv"
                output_file = f"{data_path}/output_2023_w{week_str}.csv"

                # Load real data if path provided
                self.input_data = pd.read_csv(input_file)
                self.output_data = pd.read_csv(output_file)
                self.data_loaded = True
            else:
                # Generate synthetic data for demonstration
                self.input_data = self.generate_synthetic_data()
                self.output_data = self.generate_synthetic_output()
                self.data_loaded = True

            print(f"✓ Data loaded for Week {week:02d}: {len(self.input_data)} records")
            return True

        except Exception as e:
            print(f"⚠ Using synthetic data (failed to load week {week:02d}): {e}")
            self.input_data = self.generate_synthetic_data()
            self.output_data = self.generate_synthetic_output()
            self.data_loaded = True
            return True

In [ ]:
#advance plotly 3d
class PlotlyAdvanced3D:
    """Advanced Plotly 3D visualizations"""
    
    def __init__(self, data_processor):
        self.processor = data_processor
        self.colors = {
            'field': '#2E7D32',
            'offense': '#1E88E5',
            'defense': '#DC143C',
            'ball': '#FFD700'
        }
    
    def create_animated_play(self):
        """Create advanced animated 3D play visualization"""
        
        # Get data
        data = self.processor.input_data
        
        # Create figure with subplots
        fig = make_subplots(
            rows=2, cols=2,
            specs=[[{'type': 'scatter3d', 'rowspan': 2}, {'type': 'scatter'}],
                   [None, {'type': 'scatter'}]],
            subplot_titles=('3D Field View', 'Speed Analysis', 'Separation Metrics'),
            column_widths=[0.7, 0.3],
            row_heights=[0.5, 0.5]
        )
         # Create field
        field = self._create_field_mesh()
        fig.add_trace(field, row=1, col=1)
        
        # Animate players
        frames = []
        for frame_id in data['frame_id'].unique()[:30]:
            frame_data = data[data['frame_id'] == frame_id]
            frame = self._create_frame(frame_data, frame_id)
            frames.append(frame)
        
        fig.frames = frames
        
        # Add initial data
        initial = data[data['frame_id'] == 0]
        
        # Offense players
        offense = initial[initial['player_side'] == 'Offense']
        fig.add_trace(go.Scatter3d(
            x=offense['x'], y=offense['y'], z=[2] * len(offense),
            mode='markers+text',
            marker=dict(size=10, color=self.colors['offense']),
            text=offense['player_position'],
            name='Offense'
        ), row=1, col=1)
        # Defense players
        defense = initial[initial['player_side'] == 'Defense']
        fig.add_trace(go.Scatter3d(
            x=defense['x'], y=defense['y'], z=[2] * len(defense),
            mode='markers+text',
            marker=dict(size=10, color=self.colors['defense']),
            text=defense['player_position'],
            name='Defense'
        ), row=1, col=1)
        
        # Add speed graph
        speed_data = data.groupby('frame_id')['s'].mean()
        fig.add_trace(go.Scatter(
            x=speed_data.index, y=speed_data.values,
            mode='lines', name='Avg Speed',
            line=dict(color='blue', width=2)
        ), row=1, col=2)
        
        # Add separation graph
        separation_data = []
        for frame in data['frame_id'].unique()[:30]:
            frame_data = data[data['frame_id'] == frame]
            sep = self._calculate_frame_separation(frame_data)
            separation_data.append(sep)
        
        fig.add_trace(go.Scatter(
            x=list(range(len(separation_data))), y=separation_data,
            mode='lines', name='Min Separation',
            line=dict(color='green', width=2)
        ), row=2, col=2)
        # Update layout
        fig.update_layout(
            scene=dict(
                xaxis=dict(range=[0, 120], title='Field Length'),
                yaxis=dict(range=[0, 53.3], title='Field Width'),
                zaxis=dict(range=[0, 20], title='Height'),
                camera=dict(eye=dict(x=1.5, y=-1.5, z=0.8)),
                aspectratio=dict(x=2.2, y=1, z=0.4)
            ),
            updatemenus=[dict(
                type='buttons',
                showactive=False,
                buttons=[
                    dict(label='Play', method='animate',
                         args=[None, {'frame': {'duration': 50}, 'fromcurrent': True}]),
                    dict(label='Pause', method='animate',
                         args=[[None], {'frame': {'duration': 0}, 'mode': 'immediate'}])
                ]
            )],
            height=800,
            title='NFL Advanced 3D Play Visualization'
        )
        
        return fig
    def _create_field_mesh(self):
        """Create 3D field mesh"""
        x = np.linspace(0, 120, 50)
        y = np.linspace(0, 53.3, 20)
        X, Y = np.meshgrid(x, y)
        Z = np.zeros_like(X)
        
        # Create stripe pattern
        colors = np.zeros_like(X)
        for i in range(len(x)):
            colors[:, i] = (i // 5) % 2
        
        return go.Surface(
            x=X, y=Y, z=Z,
            surfacecolor=colors,
            colorscale=['#2E7D32', '#3E8E41'],
            showscale=False
        )
    def _create_frame(self, frame_data, frame_id):
        """Create animation frame"""
        offense = frame_data[frame_data['player_side'] == 'Offense']
        defense = frame_data[frame_data['player_side'] == 'Defense']
        
        return go.Frame(
            data=[
                go.Scatter3d(x=offense['x'], y=offense['y'], z=[2] * len(offense)),
                go.Scatter3d(x=defense['x'], y=defense['y'], z=[2] * len(defense))
            ],
            name=str(frame_id)
        )
    
    def _calculate_frame_separation(self, frame_data):
        """Calculate minimum separation for frame"""
        targeted = frame_data[frame_data['player_role'] == 'Targeted Receiver']
        if not targeted.empty:
            target_x = targeted['x'].iloc[0]
            target_y = targeted['y'].iloc[0]
            
            defenders = frame_data[frame_data['player_side'] == 'Defense']
            if not defenders.empty:
                distances = np.sqrt((defenders['x'] - target_x)**2 + 
                                  (defenders['y'] - target_y)**2)
                return distances.min()
        return 10.0

In [ ]:
#pyvista sector
if PYVISTA_AVAILABLE:
    class PyVista3D:
        """PyVista-based 3D visualization"""
        
        def __init__(self, data_processor):
            self.processor = data_processor
            
        def create_interactive_scene(self):
            """Create interactive 3D scene with PyVista"""
            
            # Create plotter
            plotter = pv.Plotter(notebook=False, shape=(1, 2))
            
            # Add field
            field = pv.Plane(center=(60, 26.65, 0), 
                            direction=(0, 0, 1),
                            i_size=120, j_size=53.3)
            plotter.add_mesh(field, color='green', opacity=0.8)
            
            # Add yard lines
            for yard in range(10, 111, 10):
                line = pv.Line((yard, 0, 0.1), (yard, 53.3, 0.1))
                plotter.add_mesh(line, color='white', line_width=3)
            
            # Add players as spheres
            data = self.processor.input_data[self.processor.input_data['frame_id'] == 0]
            for _, player in data.iterrows():
                if player['player_side'] == 'Offense':
                    color = 'blue'
                else:
                    color = 'red'
                
                # Player body (cylinder)
                cylinder = pv.Cylinder(center=(player['x'], player['y'], 1),
                                      direction=(0, 0, 1),
                                      radius=0.5, height=2)
                plotter.add_mesh(cylinder, color=color)
                
                # Player head (sphere)
                sphere = pv.Sphere(center=(player['x'], player['y'], 2.5), radius=0.3)
                plotter.add_mesh(sphere, color=color)
            
            # Add ball trajectory
            t = np.linspace(0, 3, 50)
            x = 35 + t * 25
            y = 26.65 - t * 5
            z = 2 + t * 5 - t**2 * 2
            
            points = np.column_stack([x, y, z])
            spline = pv.Spline(points)
            plotter.add_mesh(spline, color='yellow', line_width=5)
            
            # Set camera
            plotter.camera_position = [(60, -50, 30), (60, 26.65, 0), (0, 0, 1)]
            # Add second subplot for metrics
            plotter.subplot(0, 1)
            
            # Create metrics visualization
            metrics_mesh = pv.PolyData(np.random.rand(100, 3) * [10, 10, 5])
            plotter.add_mesh(metrics_mesh, scalars=np.random.rand(100),
                            cmap='viridis', point_size=10)
            
            plotter.add_text('Player Metrics', position='upper_left')
            
            return plotter

In [ ]:
# vispy
if VISPY_AVAILABLE:
    class Vispy3D:
        """Vispy-based real-time 3D visualization"""
        
        def __init__(self, data_processor):
            self.processor = data_processor
            
        def create_realtime_visualization(self):
            """Create real-time 3D visualization with Vispy"""
            
            # Create canvas
            canvas = scene.SceneCanvas(keys='interactive', bgcolor='black',
                                      size=(1200, 800), show=True)
            
            # Create view
            view = canvas.central_widget.add_view()
            
            # Add field
            field_vertices = np.array([
                [0, 0, 0], [120, 0, 0], [120, 53.3, 0], [0, 53.3, 0]
            ])
            field = scene.visuals.Polygon(field_vertices[:, :2], color='green',
                                         border_color='white', parent=view.scene)
            
            # Add yard lines
            for yard in range(10, 111, 10):
                line = scene.visuals.Line(pos=np.array([[yard, 0, 0.1], 
                                                        [yard, 53.3, 0.1]]),
                                         color='white', width=2, parent=view.scene)
            # Add players
            data = self.processor.input_data[self.processor.input_data['frame_id'] == 0]
            
            offense_pos = data[data['player_side'] == 'Offense'][['x', 'y']].values
            offense_pos = np.column_stack([offense_pos, np.ones(len(offense_pos)) * 2])
            
            defense_pos = data[data['player_side'] == 'Defense'][['x', 'y']].values
            defense_pos = np.column_stack([defense_pos, np.ones(len(defense_pos)) * 2])
            
            # Add player markers
            offense_markers = scene.visuals.Markers(pos=offense_pos, size=10,
                                                   face_color='blue',
                                                   edge_color='white',
                                                   parent=view.scene)
            
            defense_markers = scene.visuals.Markers(pos=defense_pos, size=10,
                                                   face_color='red',
                                                   edge_color='white',
                                                   parent=view.scene)
            
            # Add camera
            cam = scene.TurntableCamera(elevation=30, azimuth=45, 
                                       distance=150, fov=60)
            view.camera = cam
            # Animation
        def update(event):
            # Rotate camera
            cam.azimuth += 0.5
            canvas.update()
            
            timer = scene.Timer(interval=0.03, connect=update, start=True)
            
            return canvas

In [ ]:
#advance matplotlib
class MatplotlibAdvanced3D:
    """Advanced Matplotlib 3D visualization"""
    
    def __init__(self, data_processor):
        self.processor = data_processor
        
    def create_multi_angle_view(self):
        """Create multi-angle 3D visualization"""
        
        fig = plt.figure(figsize=(20, 12))
        
        # Different camera angles
        angles = [
            {'elev': 20, 'azim': -60, 'title': 'Broadcast View'},
            {'elev': 90, 'azim': 0, 'title': 'Overhead View'},
            {'elev': 5, 'azim': -90, 'title': 'Sideline View'},
            {'elev': 15, 'azim': 0, 'title': 'End Zone View'}
        ]
        
        data = self.processor.input_data[self.processor.input_data['frame_id'] == 0]
        
        for i, angle in enumerate(angles, 1):
            ax = fig.add_subplot(2, 2, i, projection='3d')
            
            # Draw field
            self._draw_field_3d(ax)
             # Draw players
            self._draw_players_3d(ax, data)
            
            # Draw ball trajectory
            self._draw_trajectory_3d(ax)
            
            # Set view angle
            ax.view_init(elev=angle['elev'], azim=angle['azim'])
            ax.set_title(angle['title'], fontsize=14, fontweight='bold')
            
            # Remove axes for cleaner look
            ax.set_xticks([])
            ax.set_yticks([])
            ax.set_zticks([])
            
        plt.suptitle('NFL Play - Multiple 3D Perspectives', fontsize=16, fontweight='bold')
        plt.tight_layout()
        
        return fig
    def _draw_field_3d(self, ax):
        """Draw 3D field"""
        # Field surface
        x = np.linspace(0, 120, 10)
        y = np.linspace(0, 53.3, 10)
        X, Y = np.meshgrid(x, y)
        Z = np.zeros_like(X)
        
        ax.plot_surface(X, Y, Z, color='green', alpha=0.7)
        
        # Yard lines
        for yard in range(10, 111, 10):
            ax.plot([yard, yard], [0, 53.3], [0.1, 0.1], 'w-', linewidth=2)
        
        # End zones
        ax.plot_surface(X[:, :1], Y[:, :1], Z[:, :1], color='darkblue', alpha=0.5)
        ax.plot_surface(X[:, -1:], Y[:, -1:], Z[:, -1:], color='darkred', alpha=0.5)
        
        ax.set_xlim(0, 120)
        ax.set_ylim(0, 53.3)
        ax.set_zlim(0, 15)
    def _draw_players_3d(self, ax, data):
        """Draw 3D players"""
        for _, player in data.iterrows():
            if player['player_side'] == 'Offense':
                color = 'blue'
            else:
                color = 'red'
            
            # Player cylinder
            theta = np.linspace(0, 2*np.pi, 20)
            z = np.linspace(0, 2, 10)
            theta_grid, z_grid = np.meshgrid(theta, z)
            
            x_cylinder = player['x'] + 0.3 * np.cos(theta_grid)
            y_cylinder = player['y'] + 0.3 * np.sin(theta_grid)
            
            ax.plot_surface(x_cylinder, y_cylinder, z_grid, color=color, alpha=0.8)
            
            # Player sphere (head)
            u = np.linspace(0, 2*np.pi, 10)
            v = np.linspace(0, np.pi, 10)
            x_sphere = player['x'] + 0.2 * np.cos(u) * np.sin(v)[:, np.newaxis]
            y_sphere = player['y'] + 0.2 * np.sin(u) * np.sin(v)[:, np.newaxis]
            z_sphere = 2.3 + 0.2 * np.cos(v)[:, np.newaxis]
            
            ax.plot_surface(x_sphere, y_sphere, z_sphere, color=color, alpha=0.85)
    def _draw_trajectory_3d(self, ax):
            """Draw ball trajectory"""
            t = np.linspace(0, 3, 100)
            x = 35 + t * 25
            y = 26.65 - t * 5
            z = 2 + t * 8 - t**2 * 3
            z = np.maximum(z, 0)
            
            ax.plot(x, y, z, 'gold', linewidth=3, label='Ball Path')
            
            # Add markers along path
            for i in range(0, len(t), 10):
                ax.scatter(x[i], y[i], z[i], c='yellow', s=50, edgecolors='orange')

In [ ]:
#visual regulaizer
class NFLAdvanced3DSystem:
    """Main system coordinating all visualization methods"""
    
    def __init__(self):
        self.processor = NFLDataProcessor()
        self.processor.load_data()
        
        self.plotly_viz = PlotlyAdvanced3D(self.processor)
        self.matplotlib_viz = MatplotlibAdvanced3D(self.processor)
        
        if PYVISTA_AVAILABLE:
            self.pyvista_viz = PyVista3D(self.processor)
        
        if VISPY_AVAILABLE:
            self.vispy_viz = Vispy3D(self.processor)
    
    def create_all_visualizations(self):
        """Create all available visualizations"""
        
        print("\n📊 Creating Advanced 3D Visualizations...")
        
        visualizations = {}
        # 1. Plotly Interactive
        print("\n1. Creating Plotly Interactive 3D...")
        try:
            plotly_fig = self.plotly_viz.create_animated_play()
            plotly_fig.write_html("nfl_plotly_advanced.html")
            visualizations['plotly'] = plotly_fig
            print("   ✓ Saved: nfl_plotly_advanced.html")
        except Exception as e:
            print(f"   ⚠ Plotly error: {e}")
        
        # 2. Matplotlib Multi-angle
        print("\n2. Creating Matplotlib Multi-angle Views...")
        try:
            matplotlib_fig = self.matplotlib_viz.create_multi_angle_view()
            matplotlib_fig.savefig("nfl_matplotlib_multiview.png", dpi=150, bbox_inches='tight')
            visualizations['matplotlib'] = matplotlib_fig
            print("   ✓ Saved: nfl_matplotlib_multiview.png")
        except Exception as e:
            print(f"   ⚠ Matplotlib error: {e}")
        # 3. PyVista (if available)
        if PYVISTA_AVAILABLE:
            print("\n3. Creating PyVista Interactive Scene...")
            try:
                pyvista_plotter = self.pyvista_viz.create_interactive_scene()
                pyvista_plotter.show(screenshot='nfl_pyvista.png')
                visualizations['pyvista'] = pyvista_plotter
                print("   ✓ Saved: nfl_pyvista.png")
            except Exception as e:
                print(f"   ⚠ PyVista error: {e}")
        
        # 4. Vispy (if available)
        if VISPY_AVAILABLE:
            print("\n4. Creating Vispy Real-time Visualization...")
            try:
                vispy_canvas = self.vispy_viz.create_realtime_visualization()
                visualizations['vispy'] = vispy_canvas
                print("   ✓ Created Vispy real-time view")
            except Exception as e:
                print(f"   ⚠ Vispy error: {e}")
                 # Calculate and display metrics
        print("\n📈 Calculating Advanced Metrics...")
        metrics = self.processor.calculate_metrics()
        
        print("\n📊 Key Metrics:")
        print(f"   • Separation Score: {metrics['separation_score']:.2f} yards")
        print(f"   • Convergence Velocity: {metrics['convergence_velocity']:.2f} yards/frame")
        print(f"   • Route Efficiency: {metrics['route_efficiency']:.3f}")
        
        return visualizations
    def create_comparison_dashboard(self):
        """Create a dashboard comparing different visualization methods"""
        
        from plotly.subplots import make_subplots
        
        fig = make_subplots(
            rows=3, cols=2,
            subplot_titles=('Method Comparison', 'Performance Metrics',
                          'Field Heatmap', 'Player Trajectories',
                          '3D Separation Analysis', 'Speed Distribution'),
            specs=[[{'type': 'bar'}, {'type': 'scatter'}],
                   [{'type': 'heatmap'}, {'type': 'scatter3d'}],
                   [{'type': 'scatter3d'}, {'type': 'histogram'}]]
        )
        
        # Method comparison
        methods = ['Plotly', 'Matplotlib', 'PyVista', 'Vispy']
        scores = [9, 7, 8, 9]
        fig.add_trace(go.Bar(x=methods, y=scores, name='Quality Score'), row=1, col=1)
        
        # Performance metrics
        frames = list(range(30))
        performance = [np.sin(x/5) * 10 + 20 for x in frames]
        fig.add_trace(go.Scatter(x=frames, y=performance, mode='lines', name='FPS'), row=1, col=2)
        
        # Field heatmap
        heatmap_data = np.random.randn(10, 20)
        fig.add_trace(go.Heatmap(z=heatmap_data, colorscale='RdYlGn'), row=2, col=1)
        # Player trajectories
        data = self.processor.input_data
        sample = data[data['frame_id'] < 5]
        fig.add_trace(go.Scatter3d(
            x=sample['x'], y=sample['y'], z=sample['frame_id'],
            mode='markers+lines',
            marker=dict(size=5, color=sample['frame_id'], colorscale='Viridis')
        ), row=2, col=2)
        
        # 3D Separation
        offense = data[data['player_side'] == 'Offense'].head(11)
        defense = data[data['player_side'] == 'Defense'].head(11)
        fig.add_trace(go.Scatter3d(
            x=offense['x'], y=offense['y'], z=[2]*len(offense),
            mode='markers', marker=dict(color='blue', size=8),
            name='Offense'
        ), row=3, col=1)
        fig.add_trace(go.Scatter3d(
            x=defense['x'], y=defense['y'], z=[2]*len(defense),
            mode='markers', marker=dict(color='red', size=8),
            name='Defense'
        ), row=3, col=1)
        # Speed distribution
        fig.add_trace(go.Histogram(x=data['s'], nbinsx=30, name='Speed'), row=3, col=2)
        
        fig.update_layout(height=1200, showlegend=True, 
                         title_text="NFL 3D Visualization Methods Dashboard")
        
        return fig

In [ ]:
#runing the advance 3d
class NFLAdvanced3DSystem:
    """Main system coordinating all visualization methods"""
    
    def __init__(self):
        self.processor = NFLDataProcessor()
        self.processor.load_data()
        
        self.plotly_viz = PlotlyAdvanced3D(self.processor)
        self.matplotlib_viz = MatplotlibAdvanced3D(self.processor)
        
        if PYVISTA_AVAILABLE:
            self.pyvista_viz = PyVista3D(self.processor)
        
        if VISPY_AVAILABLE:
            self.vispy_viz = Vispy3D(self.processor)
    
    def create_all_visualizations(self):
        """Create all available visualizations"""
        
        print("\n📊 Creating Advanced 3D Visualizations...")
        
        visualizations = {}
        
        # 1. Plotly Interactive
        print("\n1. Creating Plotly Interactive 3D...")
        try:
            plotly_fig = self.plotly_viz.create_animated_play()
            plotly_fig.write_html("nfl_plotly_advanced.html")
            visualizations['plotly'] = plotly_fig
            print("   ✓ Saved: nfl_plotly_advanced.html")
        except Exception as e:
            print(f"   ⚠ Plotly error: {e}")
        
        # 2. Matplotlib Multi-angle
        print("\n2. Creating Matplotlib Multi-angle Views...")
        try:
            matplotlib_fig = self.matplotlib_viz.create_multi_angle_view()
            matplotlib_fig.savefig("nfl_matplotlib_multiview.png", dpi=150, bbox_inches='tight')
            visualizations['matplotlib'] = matplotlib_fig
            print("   ✓ Saved: nfl_matplotlib_multiview.png")
        except Exception as e:
            print(f"   ⚠ Matplotlib error: {e}")
        
        # 3. PyVista (if available)
        if PYVISTA_AVAILABLE:
            print("\n3. Creating PyVista Interactive Scene...")
            try:
                pyvista_plotter = self.pyvista_viz.create_interactive_scene()
                pyvista_plotter.show(screenshot='nfl_pyvista.png')
                visualizations['pyvista'] = pyvista_plotter
                print("   ✓ Saved: nfl_pyvista.png")
            except Exception as e:
                print(f"   ⚠ PyVista error: {e}")
        # 4. Vispy (if available)
        if VISPY_AVAILABLE:
            print("\n4. Creating Vispy Real-time Visualization...")
            try:
                vispy_canvas = self.vispy_viz.create_realtime_visualization()
                visualizations['vispy'] = vispy_canvas
                print("   ✓ Created Vispy real-time view")
            except Exception as e:
                print(f"   ⚠ Vispy error: {e}")
        
        # Calculate and display metrics
        print("\n📈 Calculating Advanced Metrics...")
        metrics = self.processor.calculate_metrics()
        
        print("\n📊 Key Metrics:")
        print(f"   • Separation Score: {metrics['separation_score']:.2f} yards")
        print(f"   • Convergence Velocity: {metrics['convergence_velocity']:.2f} yards/frame")
        print(f"   • Route Efficiency: {metrics['route_efficiency']:.3f}")
        
        return visualizations
    def create_comparison_dashboard(self):
        """Create a dashboard comparing different visualization methods"""
        
        from plotly.subplots import make_subplots
        
        fig = make_subplots(
            rows=3, cols=2,
            subplot_titles=('Method Comparison', 'Performance Metrics',
                          'Field Heatmap', 'Player Trajectories',
                          '3D Separation Analysis', 'Speed Distribution'),
            specs=[[{'type': 'bar'}, {'type': 'scatter'}],
                   [{'type': 'heatmap'}, {'type': 'scatter3d'}],
                   [{'type': 'scatter3d'}, {'type': 'histogram'}]]
        )
        
        # Method comparison
        methods = ['Plotly', 'Matplotlib', 'PyVista', 'Vispy']
        scores = [9, 7, 8, 9]
        fig.add_trace(go.Bar(x=methods, y=scores, name='Quality Score'), row=1, col=1)
        
        # Performance metrics
        frames = list(range(30))
        performance = [np.sin(x/5) * 10 + 20 for x in frames]
        fig.add_trace(go.Scatter(x=frames, y=performance, mode='lines', name='FPS'), row=1, col=2)
         # Field heatmap
        heatmap_data = np.random.randn(10, 20)
        fig.add_trace(go.Heatmap(z=heatmap_data, colorscale='RdYlGn'), row=2, col=1)
        
        # Player trajectories
        data = self.processor.input_data
        sample = data[data['frame_id'] < 5]
        fig.add_trace(go.Scatter3d(
            x=sample['x'], y=sample['y'], z=sample['frame_id'],
            mode='markers+lines',
            marker=dict(size=5, color=sample['frame_id'], colorscale='Viridis')
        ), row=2, col=2)
        
        # 3D Separation
        offense = data[data['player_side'] == 'Offense'].head(11)
        defense = data[data['player_side'] == 'Defense'].head(11)
        fig.add_trace(go.Scatter3d(
            x=offense['x'], y=offense['y'], z=[2]*len(offense),
            mode='markers', marker=dict(color='blue', size=8),
            name='Offense'
        ), row=3, col=1)
        fig.add_trace(go.Scatter3d(
            x=defense['x'], y=defense['y'], z=[2]*len(defense),
            mode='markers', marker=dict(color='red', size=8),
            name='Defense'
        ), row=3, col=1)
        
         # Speed distribution
        fig.add_trace(go.Histogram(x=data['s'], nbinsx=30, name='Speed'), row=3, col=2)
        
        fig.update_layout(height=1200, showlegend=True, 
                         title_text="NFL 3D Visualization Methods Dashboard")
        
        return fig

In [ ]:
#main execution
def main():
    """Main execution function"""
    
    print("\n" + "="*80)
    print(" "*20 + "STARTING NFL 3D VISUALIZATION SYSTEM")
    print("="*80)
    
    # Create system
    system = NFLAdvanced3DSystem()
    
    # Generate all visualizations
    visualizations = system.create_all_visualizations()
    
    # Create comparison dashboard
    print("\n📊 Creating Comparison Dashboard...")
    dashboard = system.create_comparison_dashboard()
    dashboard.write_html("nfl_3d_dashboard.html")
    print("   ✓ Saved: nfl_3d_dashboard.html")
    
    # Summary
    print("\n" + "="*80)
    print(" "*25 + "VISUALIZATION COMPLETE")
    print("="*80)
    
    print("\n✅ Successfully created:")
    print("   • Plotly interactive 3D with animation")
    print("   • Matplotlib multi-angle views")
    if PYVISTA_AVAILABLE:
        print("   • PyVista interactive scene")
    if VISPY_AVAILABLE:
        print("   • Vispy real-time visualization")
    print("   • Comparison dashboard")
    
    print("\n📁 Output files:")
    print("   • nfl_plotly_advanced.html")
    print("   • nfl_matplotlib_multiview.png")
    print("   • nfl_3d_dashboard.html")
    
    return visualizations

In [ ]:
#final adjustment
if __name__ == "__main__":
    visualizations = main()